## import thư viện

In [ ]:
!pip install datasets transformers accelerate

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset, Dataset
from sentence_transformers import CrossEncoder

In [ ]:
# Kiểm tra GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda


## import dataset

In [ ]:
dataset = load_dataset("anti-ai/ViNLI-Zalo-supervised")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/133 [00:00<?, ?B/s]

law_vi.jsonl.gz:   0%|          | 0.00/37.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/32980 [00:00<?, ? examples/s]

In [ ]:
print(dataset['train'][0:5])

{'query': ['Tổ sát_hạch cấp giấy_phép lái tàu_hỏa có bao_nhiêu thành_viên ?', 'Văn_bản điều_chỉnh hoạt_động tổ_chức đào_tạo nghề cho lao_động trong doanh_nghiệp nhỏ và vừa được quy_định như thế_nào ?', 'Phó chỉ_huy_trưởng Ban chỉ_huy quân_sự xã được hưởng chế_độ phụ_cấp đặc_thù như thế_nào ?', 'Thực_hiện dân_chủ trong cơ_sở giáo_dục công_lập thì người đứng đầu đoàn_thể có những trách_nhiệm nào ?', 'Mức phạt nếu điều_khiển xe ô_tô lắp thêm đèn'], 'positive': ['Điều 30 . Tổ sát_hạch 1 . Tổ sát_hạch do Cục_trưởng Cục Đường_sắt Việt_Nam thành_lập , chịu sự chỉ_đạo trực_tiếp của Hội_đồng sát_hạch . \n 2 . Tổ sát_hạch có ít_nhất 05 thành_viên , bao_gồm tổ_trưởng , các sát_hạch viên lý_thuyết và sát_hạch viên thực_hành . Tổ_trưởng Tổ sát_hạch là công_chức Cục Đường_sắt Việt_Nam hoặc lãnh_đạo doanh_nghiệp có thí_sinh dự kỳ sát_hạch , các sát_hạch viên là người đang công_tác tại doanh_nghiệp có thí_sinh tham_dự kỳ sát_hạch và người đang công_tác tại các cơ_sở đào_tạo liên_quan đến lái tàu . \n 

## cấu trúc lại dataset

In [ ]:
def restructure_data(split):
    query_list = dataset[split]["query"]
    positives = dataset[split]["positive"]
    negatives = dataset[split]["hard_neg"]

    samples = []
    for q, pos, neg in zip(query_list, positives, negatives):
        samples.append({"query": q, "document": pos, "label": 1})
        samples.append({"query": q, "document": neg, "label": 0})
    random.shuffle(samples)
    return Dataset.from_list(samples)

In [ ]:
train_full = restructure_data("train")

## chia train/dev/test

In [ ]:
split = train_full.train_test_split(test_size=0.1, seed=42)
temp_train = split["train"]
test_dataset = split["test"]

train_valid = temp_train.train_test_split(test_size=0.1, seed=42)
train_dataset = train_valid["train"]
valid_dataset = train_valid["test"]

In [ ]:
print("Train:", len(train_dataset))
print("Valid:", len(valid_dataset))
print("Test:", len(test_dataset))

Train: 53427
Valid: 5937
Test: 6596


## Load model


In [ ]:
models = [
    "cross-encoder/ms-marco-MiniLM-L6-v2",
    "Data-Lab/bge-reranker-v2-m3-cross-encoder-v0.1",
    "jinaai/jina-reranker-v2-base-multilingual",
    "Alibaba-NLP/gte-multilingual-reranker-base",
    "BAAI/bge-reranker-base",
    "huynhdat543/VietNamese_law_rerank",
]

In [ ]:
model_store = {}

print("Loading model...\n")

for name in models:
    try:
        print(f"Loading {name} ...")
        tokenizer = AutoTokenizer.from_pretrained(name)
        model = AutoModelForSequenceClassification.from_pretrained(name).to(device)
        model.eval()

        model_store[name] = {
            "tokenizer": tokenizer,
            "model": model
        }
        print(f"Loaded {name}\n")

    except Exception as e:
        print(f"Lỗi khi load model {name}: {e}")

print("Tải xong tất cả model\n")

Loading model...

Loading cross-encoder/ms-marco-MiniLM-L6-v2 ...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

...

The repository huynhdat543/VietNamese_law_rerank contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/huynhdat543/VietNamese_law_rerank .
 You can inspect the repository content at https://hf.co/huynhdat543/VietNamese_law_rerank.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


modeling.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/huynhdat543/VietNamese_law_rerank:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Loaded huynhdat543/VietNamese_law_rerank

Tải xong tất cả model



## Evaluation

In [ ]:
def batched_predict(model, tokenizer, pairs, batch_size=1):
    model.eval()
    scores = []

    with torch.no_grad():
        for i in range(0, len(pairs), batch_size):
            batch = pairs[i:i + batch_size]
            queries = [q for q, d in batch]
            docs = [d for q, d in batch]

            encoded = tokenizer(
                queries,
                docs,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors="pt"
            ).to(model.device)

            logits = model(**encoded).logits

            if logits.shape[-1] == 1:
                batch_scores = torch.sigmoid(logits).squeeze().tolist()
            else:
                batch_scores = torch.softmax(logits, dim=1)[:, 1].tolist()

            scores.extend(batch_scores)

            torch.cuda.empty_cache()

    return scores

In [ ]:
def evaluate_loaded_model(model_name, model_dict, test_dataset, batch_size=1):
    print(f"\n--- Evaluating model: {model_name} ---")

    model = model_dict["model"]
    tokenizer = model_dict["tokenizer"]

    test_pairs = list(zip(test_dataset["query"], test_dataset["document"]))
    true_labels = test_dataset["label"]

    pred_scores = batched_predict(model, tokenizer, test_pairs, batch_size=batch_size)
    pred_labels = [1 if s >= 0.5 else 0 for s in pred_scores]

    acc = accuracy_score(true_labels, pred_labels)
    f1 = f1_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels)
    recall = recall_score(true_labels, pred_labels)

    print("Classification Evaluation:")
    print(f"Accuracy:  {acc:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")

    return {
        "model": model_name,
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }


In [ ]:
results = []

for name, model_dict in model_store.items():
    try:
        res = evaluate_loaded_model(name, model_dict, test_dataset, batch_size=8)
        results.append(res)
    except Exception as e:
        print(f"Lỗi khi evaluate model {name}: {e}")

pd.DataFrame(results)


--- Evaluating model: cross-encoder/ms-marco-MiniLM-L6-v2 ---
Classification Evaluation:
Accuracy:  0.7526
F1 Score:  0.7999
Precision: 0.6780
Recall:    0.9752

--- Evaluating model: Data-Lab/bge-reranker-v2-m3-cross-encoder-v0.1 ---
Classification Evaluation:
Accuracy:  0.9295
F1 Score:  0.9336
Precision: 0.8930
Recall:    0.9782

--- Evaluating model: jinaai/jina-reranker-v2-base-multilingual ---
Classification Evaluation:
Accuracy:  0.8749
F1 Score:  0.8593
Precision: 0.9996
Recall:    0.7536

--- Evaluating model: Alibaba-NLP/gte-multilingual-reranker-base ---
Classification Evaluation:
Accuracy:  0.9189
F1 Score:  0.9196
Precision: 0.9239
Recall:    0.9154

--- Evaluating model: BAAI/bge-reranker-base ---
Classification Evaluation:
Accuracy:  0.8076
F1 Score:  0.7659
Precision: 0.9995
Recall:    0.6208

--- Evaluating model: huynhdat543/VietNamese_law_rerank ---
Classification Evaluation:
Accuracy:  0.9991
F1 Score:  0.9991
Precision: 0.9982
Recall:    1.0000


,model,accuracy,f1,precision,recall
0,cross-encoder/ms-marco-MiniLM-L6-v2,0.752577,0.799853,0.677963,0.975179
1,Data-Lab/bge-reranker-v2-m3-cross-encoder-v0.1,0.929503,0.933638,0.892984,0.978170
2,jinaai/jina-reranker-v2-base-multilingual,0.874924,0.859335,0.999603,0.753589
3,Alibaba-NLP/gte-multilingual-reranker-base,0.918890,0.919633,0.923936,0.915371
4,BAAI/bge-reranker-base,0.807611,0.765910,0.999519,0.620813
5,huynhdat543/VietNamese_law_rerank,0.999090,0.999104,0.998209,1.000000
